In [1]:
from ca2_analysis import analyzeCa2Data

# NOTES:
#                           A directory name results will be created as a sub directory of wherever the input file/files are
#                           and an xlsx file will be created for each input xlsx file that contains the analyzed results.
#
# path_to_data              Can be one of 'select_file', 'select_dir, or an actual path i.e. '../test_data'
# expected_frequency_hz:    Is a guide to assist in picking peaks and troughs. 
#                           For the purposes of ca2 analysis it should be set to the pacing frquency. 
#                           if this is not provided, you should provide the two following parameters 
#                           as an estimate of the expected peak height and width.
#                           without expected_frequency_hz or the expected peak height and width
#                           it is essentially impossible to know where the peaks and troughs of interest are.
# expected_min_peak_width:  
# expected_min_peak_height:
# display_resultsL          Can be either True or False and turns on or off displaying here, a plot of each input, with
#                           the peaks and troughs highlighted and the peak to trough and trough to peak average metrics.

In [2]:
analyzeCa2Data(
    path_to_data = 'select_dir',
    expected_frequency_hz = 1.0,
    display_results = True
)